In [1]:
import numpy as np
import pandas as pd

#load csv file
df = pd.read_csv('/Users/ishratjahanananya/Documents/REPOS/TrademarkVista/uspto_db/data/csv_files/trademarks.csv')

#load only first 10 rows in new dataframe
df1 = df.head(100) 

#save the new dataframe to a new csv file
df1.to_csv('/Users/ishratjahanananya/Documents/REPOS/TrademarkVista/uspto_db/data/csv_files/trademarks_100.csv', index=False, header=False)



In [2]:
df1.head()

,category-code,mark-identification,serial-number,Case-File-Owners,status,xml_filename
0,007,PERFECT HARMONY,78636704,"INNOMOTICS GMBH, SIEMENS AKTIENGESELLSCHAFT, S...",800,apc240120.xml
1,001,THREAD MAGIC,86491524,"DPG USA, Inc., DPG USA, Inc., DPG USA, Inc.",702,apc240120.xml
2,003,SIMPLE SHINE.,86912261,"TORQUAY ETRADING LLC, Love And Pieces LLC, Lov...",710,apc240120.xml
3,032,REFRESHING HYDRATION,88642175,"Super Bakery, Inc., Super Bakery, Inc., Super ...",700,apc240120.xml
4,025,SCHOOLWEAR MADE FOR KIDS TO SHINE,88697251,"Lollytogs, Ltd., Lollytogs, Ltd.",606,apc240120.xml


In [5]:
!curl -X POST http://127.0.0.1:5000/api/query \
     -H "Content-Type: application/json" \
     -d '{"question": "Find trademarks with NTHLIFE"}'

{
  "searchMarks": [
    {
      "caseFileOwners": "Samsar Resources, LLC, Samsar Resources, LLC, Samsar Resources, LLC",
      "categoryCode": "9",
      "id": 82,
      "markIdentification": "NTHLIFE",
      "serialNumber": "97436279",
      "status": "700"
    },
    {
      "caseFileOwners": "Samsar Resources, LLC, Samsar Resources, LLC",
      "categoryCode": "9",
      "id": 83,
      "markIdentification": "NTHLIFE",
      "serialNumber": "97436322",
      "status": "688"
    },
    {
      "caseFileOwners": "Samsar Resources, LLC, Samsar Resources, LLC",
      "categoryCode": "40",
      "id": 84,
      "markIdentification": "NTHLIFE",
      "serialNumber": "97436347",
      "status": "730"
    },
    {
      "caseFileOwners": "Samsar Resources, LLC, Samsar Resources, LLC",
      "categoryCode": "40",
      "id": 85,
      "markIdentification": "NTHLIFE",
      "serialNumber": "97436361",
      "status": "730"
    }
  ]
}


In [4]:
from trademarkvista.TrademarkQA import TrademarkQA
from trademarkvista.SmolLMWrapper import SmolLMWrapper
import graphene
from sqlalchemy import create_engine, select
from sqlalchemy.orm import Session, declarative_base
from sqlalchemy import Column, Integer, String, Text
from pprint import pprint

user_question = 'Are there any trademarks with NTHLIFE?'
# Database Setup
engine = create_engine('postgresql://localhost/trademark_db')
Base = declarative_base()

# SQLAlchemy Model
class TrademarkModel(Base):
    __tablename__ = 'trademarks'
    id = Column(Integer, primary_key=True)
    category_code = Column(String)
    mark_identification = Column(Text)
    serial_number = Column(String, unique=True)
    case_file_owners = Column(Text)
    status = Column(String)
    xml_filename = Column(String)


# GraphQL Schema
class Trademark(graphene.ObjectType):
    id = graphene.Int()
    category_code = graphene.String()
    mark_identification = graphene.String()
    serial_number = graphene.String()
    case_file_owners = graphene.String()
    status = graphene.String()
    xml_filename = graphene.String()

class Query(graphene.ObjectType):
    all_trademarks = graphene.List(Trademark)
    trademark_by_serial = graphene.Field(Trademark, serial_number=graphene.String())
    trademarks_by_category = graphene.List(Trademark, category_code=graphene.String())
    search_marks = graphene.List(Trademark, keyword=graphene.String())

    def resolve_all_trademarks(self, info):
        with Session(engine) as session:
            return session.execute(select(TrademarkModel)).scalars().all()

    def resolve_trademark_by_serial(self, info, serial_number):
        with Session(engine) as session:
            return session.execute(
                select(TrademarkModel).filter_by(serial_number=serial_number)
            ).scalar_one_or_none()

    def resolve_trademarks_by_category(self, info, category_code):
        with Session(engine) as session:
            return session.execute(
                select(TrademarkModel).filter_by(category_code=category_code)
            ).scalars().all()

    def resolve_search_marks(self, info, keyword):
        with Session(engine) as session:
            return session.execute(
                select(TrademarkModel).filter(
                    TrademarkModel.mark_identification.ilike(f'%{keyword}%')
                )
            ).scalars().all()


schema = graphene.Schema(query=Query)
llm_wrapper = SmolLMWrapper()
qa_system = TrademarkQA(llm_wrapper, schema)
result = qa_system.process_query(user_question)
print('Question:', user_question)
pprint(result, indent=2)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Loaded pipeline
Loaded LLM
Extracted trademark: NTHLIFE
Extracted category: None
Generated GraphQL query: query { 
    searchMarks(keyword: "NTHLIFE") { 
        id
        markIdentification
        serialNumber
        categoryCode
        status
        caseFileOwners
    } 
}
Question: Are there any trademarks with NTHLIFE?
{ 'searchMarks': [ { 'caseFileOwners': 'Samsar Resources, LLC, Samsar '
                                       'Resources, LLC, Samsar Resources, LLC',
                     'categoryCode': '9',
                     'id': 82,
                     'markIdentification': 'NTHLIFE',
                     'serialNumber': '97436279',
                     'status': '700'},
                   { 'caseFileOwners': 'Samsar Resources, LLC, Samsar '
                                       'Resources, LLC',
                     'categoryCode': '9',
                     'id': 83,
                     'markIdentification': 'NTHLIFE',
                     'serialNumber': '97436322'